In [1]:
import torch
import torch.nn as nn

import json

import pandas as pd

from tqdm import tqdm, trange

from torch.utils.data import TensorDataset

The file "release_conditions.json" is a json-dumped dictionary that contains all the disease that a patient can suffer from. "release_evidences.json" is a json-dumped dictionary of all the possible symptoms.

The .csv files are lists of patients, the disease they suffer from, their symptoms, sex, and some other information that we will not make use of.

In [15]:
conditions_file = "release_conditions.json"
evidences_file = "release_evidences.json"
train_file= "release_train_patients/release_train_patients.csv"
test_file = "release_test_patients/release_test_patients.csv"
valid_file = "release_validate_patients/release_validate_patients.csv"

In [3]:
def read_json_file(file_path):
  with open(file_path) as file:
    data = file.read()
    return json.loads(data)

In [4]:
conditions = read_json_file(conditions_file)
evidences = read_json_file(evidences_file)

In [16]:
df_train = pd.read_csv(train_file)
df_test = pd.read_csv(test_file)
df_valid = pd.read_csv(valid_file)

In [6]:
def parse_evidence_string(evidence_string):
  json_evidence = evidence_string.replace('"', "!").replace("'", '"').replace("!", "'")
  return json.loads(json_evidence)

We augment the dictionary read from the release_evidences.json file, resulting in a new dictionary, which makes the following changes:
- a multiple-choice evidence (meaning the patient can choose more than one value) X with values Y, Z and W becomes 3 evidences, X-Y, X-Z and X-W
- categorical evidences, for which the patient is presented with a list of options from which they can select only one, remain one single evidence, and their values are encoded as positive numbers starting at 0, following the meaning of the evidence, where possible (for example the numbers associated to the intensity of pain grow as the intensity grows)
- binary evidences are left as is

The new dictionary consists of:
- the evidences/symptoms as keys
- dictionaries describing the symptoms as values
- the dictionary corresponding to a binary value contains its assigned numerical index, and its default value
- the dictionary corresponding to a categorical value contains its assigned numerical index, a dictionary that assignes numerical values to its values, and its default value, in numerical form
- the multiple-choice symptoms are discretized into binary symptoms
- each evidence dictionary also contains the english description of the symptom, and the value dictionary contain english descriptions of the values

In [7]:
def make_evidence_dict(evidences_df):
  evidence_dict = dict()
  for evidence in tqdm(evidences_df.values()):
    name = evidence["name"]
    default = evidence["default_value"]
    description = evidence["question_en"]
    type = evidence["data_type"]

    # binary data
    if type == 'B':
      evidence_dict[name] = {
        "default" : default,
        "description" : description,
        "index" : len(evidence_dict),
        "type" : "binary",
      }
      continue

    # categorical symptoms
    if type == 'C' and "value_meaning" in evidence and len(evidence["value_meaning"]) != 0:
      evidence_dict[name] = dict()

      evidence_dict[name]["values"] = dict()
      for value_name, value_data in evidence["value_meaning"].items():
        evidence_dict[name]["values"][str(value_name)] = {
          "index" : len(evidence_dict[name]["values"]),
          "description" : value_data["en"]
        }
      
      evidence_dict[name]["default"] = evidence_dict[name]["values"][str(default)]["index"]
      evidence_dict[name]["description"] = evidence["question_en"]
      evidence_dict[name]["index"] = len(evidence_dict)
      evidence_dict[name]["type"] = "categorical"
      continue

    if type == 'C' and ((not "value_meaning" in evidence) or len(evidence["value_meaning"]) == 0):
      evidence_dict[name] = dict()
      evidence_dict[name]["values"] = dict()
      for value in evidence["possible-values"]:
        evidence_dict[name]["values"][str(value)] = {
          "index" : len(evidence_dict[name]["values"]),
          "description" : value
        }

      evidence_dict[name]["default"] = int(default)
      evidence_dict[name]["description"] = description
      evidence_dict[name]["index"] = len(evidence_dict)
      evidence_dict[name]["type"] = "categorical"
      continue

    # multiple-choice value
    if type == 'M':
      for value_name, value_data in evidence["value_meaning"].items():
        discretized_name = name + "%" + value_name
        evidence_dict[discretized_name] = {
          "index" : len(evidence_dict),
          "default_value" : 1 if value_name == default else 0,
          "description" : description + " " + value_data["en"],
          "type" : "multiple"
        }
      continue

  return evidence_dict

In [8]:
def make_disease_dict(diseases):
  disease_to_numerical = dict()
  for disease in diseases:
    disease_to_numerical[disease] = len(disease_to_numerical)
  return disease_to_numerical

In [9]:
evidence_dict = make_evidence_dict(evidences)
disease_dict = make_disease_dict(conditions)

100%|██████████| 223/223 [00:00<00:00, 111468.21it/s]


In [10]:
def evidences_string_to_dict(evidences):
  # swap " and ' to turn the string into valid JSON
  evidences = evidences.replace("'", '&').replace('"', "'").replace("&", '"')
  return json.loads(evidences)

In [11]:
def df_to_Tensors(df, evidence_dict, disease_dict):
  size = len(df)
  # we also take in age and sex as features, besides the symptoms
  feature_count = len(evidence_dict)

  # X_tensor[i][j] = 1 if patient i has symptom j
  # for sex, 0 means male and 1 means female
  # for age and categorical symptoms, X_tensor[i][j] is the number that
  # represents the age/index of the value
  # the last feature is the age, the feature before that is the sex
  X_tensor = torch.zeros(size, feature_count)

  # y_tensor[i][j] = 1 if patient i has disease j, and 0 otherwise
  y_tensor = torch.zeros(size).type(torch.LongTensor)

  for index, row in tqdm(enumerate(df.itertuples(index = False))):
    age = row.AGE
    evidences = row.EVIDENCES
    sex = row.SEX
    pathology = row.PATHOLOGY

    X_tensor[index][-1] = age
    X_tensor[index][-2] = 0 if sex == 'M' else 1

    evidence_list = evidences_string_to_dict(evidences)

    for evidence in evidence_list:
      if '_@_' in evidence:
        evidence_name, evidence_value = evidence.split("_@_")

        # categorical symptom
        if evidence_name in evidence_dict:
          disease_index = evidence_dict[evidence_name]["index"]
          value_index = evidence_dict[evidence_name]["values"][evidence_value]["index"]
          X_tensor[index][disease_index] = value_index
        
        # multi symptom
        if evidence_name not in evidence_dict:
          disease_index = evidence_dict[evidence_name + "%" + evidence_value]["index"]
          X_tensor[index][disease_index] = 1
      
      else:
        # binary symptom
        X_tensor[index][evidence_dict[evidence]["index"]] = 1

      y_tensor[index] = disease_dict[pathology]

  return X_tensor, y_tensor

In [34]:
X_train, y_train = df_to_Tensors(df_train, evidence_dict, disease_dict)

1025602it [04:06, 4160.17it/s]


In [35]:
X_test, y_test = df_to_Tensors(df_test, evidence_dict, disease_dict)

0it [00:00, ?it/s]

134529it [00:33, 4058.33it/s]


In [17]:
X_valid, y_valid = df_to_Tensors(df_valid, evidence_dict, disease_dict)

132448it [00:41, 3155.91it/s]


In [42]:
with open("clean_dataset/diseases.json", "w") as file:
  file.write(json.dumps(disease_dict)) 

In [43]:
with open("clean_dataset/evidences.json", "w") as file:
  file.write(json.dumps(evidence_dict)) 

In [38]:
torch.save(X_train, "clean_dataset/X_train.pt")
torch.save(y_train, "clean_dataset/y_train.pt")

In [39]:
torch.save(X_test, "clean_dataset/X_test.pt")
torch.save(y_test, "clean_dataset/y_test.pt")

In [18]:
torch.save(X_valid, "clean_dataset/X_valid.pt")
torch.save(y_valid, "clean_dataset/y_valid.pt")

In [40]:
d={"ab":{"abc":1, "bf":2}}

In [41]:
d["ab"].keys()

dict_keys(['abc', 'bf'])

In [19]:
for key, value in tqdm(evidence_dict.items(), position = 0, leave = True):
  if "short" in list(value.keys()):
    continue
  print(value["description"], flush = True)
  short = input()
  evidence_dict[key]["short"] = short
  print(short, flush = True)

  0%|          | 0/894 [00:00<?, ?it/s]

Have you recently had numbness, loss of sensation or tingling, in both arms and legs and around your mouth?
Numbness/loss of sensation/tingling in arms, legs and around mouth


 84%|████████▍ | 753/894 [00:21<00:04, 34.75it/s]

Did you previously, or do you currently, have any weakness/paralysis in one or more of your limbs or in your face?
Weakness/paralysis in limbs or face
Have you been in contact with or ate something that you have an allergy to?


 84%|████████▍ | 753/894 [00:39<00:04, 34.75it/s]

...

100%|█████████▉| 892/894 [27:46<00:12,  6.35s/it]

Are there members of your family who have been diagnosed with pancreatic cancer?
Family member diagnosed with pancreatic cancer


100%|█████████▉| 893/894 [27:58<00:08,  8.00s/it]

Have you ever had fluid in your lungs?
Had fluid in lungs


100%|██████████| 894/894 [28:04<00:00,  1.88s/it]
